In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
def clean_data(file):
    data = pd.read_csv(file)
    data = data[np.isfinite(data['DEP_TIME'])]
    data.fillna(0,inplace = True)
    return data

In [14]:
basic = "../raw_data/ontime_2014"
data_list = []
for i in xrange(1,13):
    doc = 0
    if i<10:
        doc = basic + '0' + str(i) + '.csv'
    else:
        doc = basic + str(i) + '.csv'
    print doc
    data = clean_data(doc)
    dataMean = data.groupby(['MONTH','UNIQUE_CARRIER','ORIGIN','DEST']).mean()
    #print dataMean.shape
    dataMean.reset_index(inplace = True)
    dataMerge = pd.merge(data[['MONTH','UNIQUE_CARRIER','ORIGIN','DEST', 'ORIGIN_CITY_NAME', 
                               'ORIGIN_STATE_ABR', 'DEST_CITY_NAME', 'DEST_STATE_ABR']], 
                         dataMean, on = ['MONTH','UNIQUE_CARRIER','ORIGIN','DEST'],
                         how = 'inner')
    
    # add flight volume
    flightVolume = data.groupby(['MONTH','UNIQUE_CARRIER','ORIGIN','DEST']).agg({"FL_NUM": np.count_nonzero}).reset_index()
    flightVolume.columns = ['MONTH','UNIQUE_CARRIER','ORIGIN','DEST','FLIGHT_VOLUME']
    dataMerge = pd.merge(dataMerge, flightVolume, on = ['MONTH','UNIQUE_CARRIER','ORIGIN','DEST'], how = 'inner')

    print dataMerge.shape
    data_list.append(dataMerge)
    #dataMean.to_csv('ontime_avg_2014' +str(i) +'.csv')

../raw_data/ontime_201401.csv
(441622, 24)
../raw_data/ontime_201402.csv
(407351, 24)
../raw_data/ontime_201403.csv
(494193, 24)
../raw_data/ontime_201404.csv
(478192, 24)
../raw_data/ontime_201405.csv
(490240, 24)
../raw_data/ontime_201406.csv
(492974, 24)
../raw_data/ontime_201407.csv
(512953, 24)
../raw_data/ontime_201408.csv
(501826, 24)
../raw_data/ontime_201409.csv
(463001, 24)
../raw_data/ontime_201410.csv
(486038, 24)
../raw_data/ontime_201411.csv
(457966, 24)
../raw_data/ontime_201412.csv
(470713, 24)


In [15]:
print data_list[0].columns

Index([u'MONTH', u'UNIQUE_CARRIER', u'ORIGIN', u'DEST', u'ORIGIN_CITY_NAME', u'ORIGIN_STATE_ABR', u'DEST_CITY_NAME', u'DEST_STATE_ABR', u'YEAR', u'AIRLINE_ID', u'FL_NUM', u'ORIGIN_AIRPORT_ID', u'DEST_AIRPORT_ID', u'DEP_TIME', u'DEP_DELAY', u'ARR_TIME', u'ARR_DELAY', u'CARRIER_DELAY', u'WEATHER_DELAY', u'NAS_DELAY', u'SECURITY_DELAY', u'LATE_AIRCRAFT_DELAY', u'Unnamed: 25', u'FLIGHT_VOLUME'], dtype='object')


In [16]:
data_list[0].drop_duplicates(inplace=True)
aggregateData = data_list[0]
for i in xrange(1,12):
    data_list[i].drop_duplicates(inplace=True)
    aggregateData = aggregateData.append(data_list[i])

In [18]:
aggregateData.drop(['AIRLINE_ID','DEP_TIME','ARR_TIME','FL_NUM','Unnamed: 25'], axis=1, inplace=True)
aggregateData.sort(['MONTH','UNIQUE_CARRIER','ORIGIN','DEST'], inplace=True)
print aggregateData.shape
aggregateData.to_csv('../avg_data_cleaned/ontime_avg_2014.csv', index=False)

(73544, 19)
